In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

## Summery
1. Reading the files
2. Considerations
3. FE
# 

#### 1. Reading the files

In [6]:
df = pd.read_csv('../data/data_eda.csv',parse_dates={'order_date':['date']})

In [9]:
df.dtypes

order_date               datetime64[ns]
customer_id                      object
order_hour                        int64
customer_order_rank             float64
is_failed                         int64
voucher_amount                  float64
delivery_fee                    float64
amount_paid                     float64
restaurant_id                     int64
city_id                           int64
payment_id                        int64
platform_id                       int64
transmission_id                   int64
is_returning_customer             int64
is_l2015                           bool
is_voucher_ge30                   int64
is_voucher_ge20_le30              int64
is_delivery_fee_ge5               int64
dtype: object

In [10]:
df.set_index('customer_id', inplace=True)

#### 2.Considerations

In [11]:
gp = df.groupby(df.index)